In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import base64
import sys

import solana.rpc.api

sys.path.append('..')

import settings.addresses

In [ ]:
# DOES NOT RETURN THE SAME OBJECT. Some transactions would not have data hash e.g.:
# without hash:
# Block number = 242197066.
# Timestamp = 1705424747.
# Transaction hash = ufSrdp8F61GAZ7TvM8g2hCrDVLM3NFjBjvMmdcnNMJGByg1v5mBbEyNCrb8ZWQoJsKSEEbYp2G9fePsv5YHbarQ.
# with hash:
# Block number = 242198179.
# Timestamp = 1705425225.
# Transaction hash = 4RJ4r1irMjuPdysHBNFQBnWCTE7AuZe4fMBJjivStfACkV1m2r1USW9ZXK7Xpdg9fW5JcEqRr8PggNQAgZPTHcWr.


# TODO: https://api.mainnet-beta.solana.com
solana_client = solana.rpc.api.Client("https://docs-demo.solana-mainnet.quiknode.pro/") # RPC url - now it's just some demo i found on internet


res = solana_client.get_signatures_for_address(
    solana.rpc.api.Pubkey.from_string(settings.addresses.MANGO_ADDRESS), # Address 
    limit = 100,
    # before =  # pass some signature here to fetch txs before that one (when fetching more distant history)
)

transaction = solana_client.get_transaction(
    res.value[0].signature, 
    'jsonParsed', 
    max_supported_transaction_version=0
)

In [ ]:
# Explore a single transaction.
transaction

In [ ]:
# TODO: Should we also care about 'Program logged: '?
DATA_SUBSTRING = 'Program data: '


# Extract relevant information.
print('***GENERAL***')
print(f'Block number = {transaction.value.slot}.')
print(f'Timestamp = {transaction.value.block_time}.')
print(f'Transaction hash = {transaction.value.transaction.transaction.signatures[0]}.')

print('\n***INSTRUCTIONS***')
for i, instruction in enumerate(transaction.value.transaction.transaction.message.instructions):
    # TODO: Would it be useful to filter relevant data based in these addresses?
    print(f'Instruction = {i}, address = {instruction.program_id}.')
    # TODO: This might be useful, but needs to be decoded first.
    print(f'Instruction = {i}, data = {instruction.data}.')

print('\n***META***')
# TODO: The lengths (and thus also likely the order) of the balances correspond to the length of 
# `transaction.value.transaction.transaction.message.account_keys`, but are the balances relevant?
print(f'Pre-balances = {transaction.value.transaction.meta.pre_balances}.')
print(f'Post-balances = {transaction.value.transaction.meta.post_balances}.')
data_logs = [x for x in transaction.value.transaction.meta.log_messages if DATA_SUBSTRING in x]
assert len(data_logs) == 1
data_hash = data_logs[0].replace(DATA_SUBSTRING, '')
print(f'Data hash = {data_hash}.')

In [ ]:
log_messages = transaction.value.transaction.meta.log_messages

program_data = next((i for i in log_messages if i.startswith('Program data: ')))
encoded_data = program_data.split(' ')[-1]

print(f'{DATA_SUBSTRING} {encoded_data}')

decoded_bytes = base64.b64decode(encoded_data).hex()
print(f'Decoded program data: {decoded_bytes}')